### Upload a data

`X_train` - train image dataset
`Y_train` - train label dataset
`X_test` - test image dataset
`Y_test` - test label dataset

In [1]:
import tensorflow as tf
import os
from PIL import Image
import random
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as np
from IPython.display import display

In [2]:
import json
train_json = './Data_Sources/al5083/train/train.json'

with open(train_json, 'r') as json_file:
    train_data_json = json.load(json_file)

label_count = [0, 0, 0, 0, 0, 0]
for l in train_data_json:
    label_count[train_data_json[l]] += 1

label_count

[8758, 1783, 6325, 4028, 2953, 2819]

In [35]:
# Define names
label_names = ["good weld", "burn through", "contamination", "lack of fusion", "misalignment", "lack of penetration"]

# Define labels
labels = [0, 1, 2, 3, 4, 5]

# Number of classes
num_classes = 6

# Label is converted into a binary vector where only the position corresponding to the label is set to 1, and all other positions are set to 0.
one_hot_encoding_test = [[1 if i == label else 0 for i in range(num_classes)] for label in labels]

In [6]:
all_data = []
image_size = (16, 16)
label_count = [0, 0, 0, 0]

In [7]:
import os
def load_data(path):
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if(os.path.isdir(folder_path)):
            for img_name in os.listdir(folder_path):

                img_path = os.path.join(folder_path, img_name)
                key_name = folder + '/' + img_name
                label = train_data_json[key_name]

                if label == 4 or label == 5:
                    continue
                if(label_count[label] > 100):
                    continue

                label_count[label] += 1
                image = Image.open(img_path).resize(image_size)
                #image = tf.convert_to_tensor(image)
                image = np.array(image)
                all_data.append((image, label))

                if(len(all_data)%100==0):
                    print(f"Total images loaded : {len(all_data)}, Label count : {label_count}", end = '\r')

In [8]:
load_data('./Data_Sources/al5083/train')

In [9]:
random.shuffle(all_data)

In [29]:
X = []
Y = []
for data in all_data:
    X.append(data[0])
    Y.append(data[1])

In [ ]:
import matplotlib.pyplot as plt
rand_img = np.random.randint(low = 0, high = len(X))
plt.imshow(X[rand_img])
plt.show()
print(f"Label : {label_names[Y[rand_img]]}")

Split the data into training sets: 80% of the total data for training

In [30]:
# Split the data into : 80% of the total data for training,20% of the total data for testing
total_data = len(X)
X_train, Y_train = X[:int(total_data*0.8)], Y[:int(total_data*0.8)]
X_test, Y_test = X[int(total_data*0.8):], Y[int(total_data*0.8):]

This means converting the 2D array of image pixels (16x16 pixels) into a 1D array.

In [31]:
# This means converting the 2D array of image pixels (16x16 pixels) into a 1D array.
# Reshape each image in X_train to 1D array
X_train = [x.reshape(-1) for x in X_train]

# Reshape each image in X_test to 1D array
X_test = [x.reshape(-1) for x in X_test]

Training set:

In [39]:
values, counts = np.unique( Y_train, return_counts=True)
print(values)
print(counts)

[0 1 2 3]
[83 82 82 76]


Testing set:

In [40]:
values, counts = np.unique(Y_test, return_counts=True)
print(values)
print(counts)

[0 1 2 3]
[18 19 19 25]


### Model

PEPS architecture - circuit

In [13]:
import pennylane as qml
from pennylane import numpy as np
dev = qml.device("default.qubit", wires=8)

@qml.qnode(dev)
def circuit_peps(image, template_weights):
    qml.AmplitudeEmbedding(features=image, wires=range(8), normalize=True)

    i = 0

    qml.RY(template_weights[i], wires=3)
    i = i+1
    qml.RY(template_weights[i], wires=4)
    i = i+1
    qml.CNOT(wires=[3, 4])

    qml.RY(template_weights[i], wires=2)
    i = i+1
    qml.RY(template_weights[i], wires=3)
    i = i+1
    qml.CNOT(wires=[2, 3])

    qml.RY(template_weights[i], wires=4)
    i = i+1
    qml.RY(template_weights[i], wires=5)
    i = i+1
    qml.CNOT(wires=[4, 5])

    for j in range(1, 6, 2):
        qml.RY(template_weights[i], wires=j)
        i = i+1
        qml.RY(template_weights[i], wires=j+1)
        i = i+1
        qml.CNOT(wires=[j, j+1])

    for j in range(0, 7, 2):
        qml.RY(template_weights[i], wires=j)
        i = i+1
        qml.RY(template_weights[i], wires=j+1)
        i = i+1
        qml.CNOT(wires=[j, j+1])

    for j in range(1, 6, 2):
        qml.RY(template_weights[i], wires=j)
        i = i+1
        qml.RY(template_weights[i], wires=j+1)
        i = i+1
        qml.CNOT(wires=[j, j+1])

    qml.RY(template_weights[i], wires=2)
    i = i+1
    qml.RY(template_weights[i], wires=3)
    i = i+1
    qml.CNOT(wires=[2, 3])

    qml.RY(template_weights[i], wires=4)
    i = i+1
    qml.RY(template_weights[i], wires=5)
    i = i+1
    qml.CNOT(wires=[4, 5])

    qml.RY(template_weights[i], wires=3)
    i = i+1
    qml.RY(template_weights[i], wires=4)
    i = i+1
    qml.CNOT(wires=[3, 4])


    return qml.probs(wires = [3, 4])

weights = np.random.random(size=[32, 1])
#qml.draw_mpl(circuit_peps, expansion_strategy="device")(X_train[0], weights)
#circuit_peps(X_train[0], weights)

Cost function with Cross-Entropy Loss

In [14]:
def costfunc(params):
    cost = 0
    y_true = [np.array([1, 0, 0, 0]), np.array([0, 1, 0, 0]), np.array([0, 0, 1, 0]), np.array([0, 0, 0, 1])]
    len_X_train = len(X_train)
    for i in range(len_X_train):
        prob = circuit_peps(X_train[i], params)
        if Y_train[i] == 0:
            cost -= 1/len_X_train * np.sum(np.sum(y_true[0].astype(float)*np.log(prob.astype(float))))
        elif Y_train[i] == 1:
            cost -= 1/len_X_train * np.sum(np.sum(y_true[1].astype(float)*np.log(prob.astype(float))))
        elif Y_train[i] == 2:
            cost -= 1/len_X_train * np.sum(np.sum(y_true[2].astype(float)*np.log(prob.astype(float))))
        elif Y_train[i] == 3:
            cost -= 1/len_X_train * np.sum(np.sum(y_true[3].astype(float)*np.log(prob.astype(float))))
    return cost

training the circuit parameters

In [15]:
# training the circuit parameters
x2 = np.random.random(size=[32, 1], requires_grad=True)
params = x2[:]
optimizer = qml.AdamOptimizer(stepsize=0.1)

for k in range(100):
    if k % 5 == 0:
        print(f"Step {k}, cost: {costfunc(params)}")
    params = optimizer.step(costfunc, params)
print(params)

Step 0, cost: -2.347275754598198
Step 5, cost: -2.569599898513015
Step 10, cost: -2.624928286041875
Step 15, cost: -2.64977829061237
Step 20, cost: -2.6511906009631545
Step 25, cost: -2.6572449298551586
Step 30, cost: -2.660538528761585
Step 35, cost: -2.662212348313461
Step 40, cost: -2.66391819908254
Step 45, cost: -2.6646005873995966
Step 50, cost: -2.6658621191052707
Step 55, cost: -2.666493346971844
Step 60, cost: -2.667474213461128
Step 65, cost: -2.668613727469632
Step 70, cost: -2.670033092827536
Step 75, cost: -2.6719997840126184
Step 80, cost: -2.674507715164897
Step 85, cost: -2.678259087218466
Step 90, cost: -2.682908832179458
Step 95, cost: -2.685390752089934
[[ 0.23670566]
 [-0.15176815]
 [ 1.13633239]
 [-0.0083747 ]
 [ 1.62713961]
 [ 1.60896541]
 [ 0.60639152]
 [-0.67628157]
 [ 1.65680658]
 [ 0.12393486]
 [ 0.29607796]
 [ 0.41460342]
 [ 0.08983246]
 [-0.02277222]
 [-0.27908041]
 [ 1.38748558]
 [-0.5679229 ]
 [ 1.10174569]
 [ 1.48057622]
 [-0.32357865]
 [-0.17830272]
 [ 0

In [16]:
params

tensor([[ 0.23670566],
        [-0.15176815],
        [ 1.13633239],
        [-0.0083747 ],
        [ 1.62713961],
        [ 1.60896541],
        [ 0.60639152],
        [-0.67628157],
        [ 1.65680658],
        [ 0.12393486],
        [ 0.29607796],
        [ 0.41460342],
        [ 0.08983246],
        [-0.02277222],
        [-0.27908041],
        [ 1.38748558],
        [-0.5679229 ],
        [ 1.10174569],
        [ 1.48057622],
        [-0.32357865],
        [-0.17830272],
        [ 0.81136554],
        [ 0.5335887 ],
        [ 0.62265587],
        [ 1.8043293 ],
        [ 0.9593962 ],
        [-0.23985406],
        [ 0.05755647],
        [ 0.65042842],
        [ 1.61145876],
        [-0.05241326],
        [ 0.45662746]], requires_grad=True)

### Checking the circuit on first X_test data.

In [24]:
x = circuit_peps(X_test[0], params)
np.argmax(x)

3

In [20]:
Y_test[0]

3

### Calculating accuracy
On the test set extracted from the training set. When predicted_label does not match test_label, both values are displayed.

In [27]:
test_correct = 0

for img, true_label in zip(X_test, Y_test):
    # Get predictions from the quantum circuit
    predictions = circuit_peps(img, params)

    # Determine the predicted label
    predicted_label = np.argmax(predictions)

    # Increment correct count if prediction matches the true label
    if predicted_label == true_label:
        test_correct += 1
    else:
        print("Predicted label:", predicted_label)
        print("True label",true_label)

# Calculate and print the testing accuracy percentage
accuracy = 100 * test_correct / len(X_test)
print(f"Testing accuracy: {accuracy:.2f}%")

Predicted label: 3
True label 1
Predicted label: 3
True label 1
Predicted label: 3
True label 1
Predicted label: 3
True label 1
Predicted label: 3
True label 1
Predicted label: 3
True label 1
Predicted label: 3
True label 1
Predicted label: 3
True label 1
Predicted label: 2
True label 3
Testing accuracy: 88.89%


We can see that it is most often mistaken in recognizing class 1. The model says it is 3 when in fact it is 1.

### Summary
Classification for 4 class, PEPS circuit.
Using 400 images from train dataset.
Training set: images [83 82 82 76] per classes [0 1 2 3]
Testing set: images [18 19 19 25] per classes [0,1,2,3]

Testing accuracy: 88.89%
